# Upper Air Sounding Analysis

This notebook creates Skew-T Log-P diagrams from upper air sounding data accessed via REST API.

## Setup

First, let's install the required packages:

In [ ]:
!pip install requests pandas numpy matplotlib metpy

## Import Libraries

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from metpy.plots import SkewT
from metpy.units import units
from metpy.calc import (
    parcel_profile, 
    cape_cin, 
    lifted_index, 
    precipitable_water, 
    wind_components, 
    bulk_shear
)

## Define Functions

In [ ]:
def fetch_sounding_data(report_id):
    """Fetch sounding data from the API"""
    url = f"http://65.21.25.17/collections/upper_air/items?report={report_id}&sortby=-z_value"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        data = response.json()
        features = data.get('features', [])
        
        rows = []
        for feature in features:
            props = feature.get('properties', {})
            geom = feature.get('geometry', {})
            coords = geom.get('coordinates', [0, 0])
            
            row = {
                'z_value': props.get('z_value'),
                'datetime': props.get('phenomenon_time_end'),
                'lon': coords[0],
                'lat': coords[1],
                'temperature': props.get('air_temperature'),
                'dew_point_temperature': props.get('dew_point_temperature'),
                'wind_speed': props.get('wind_speed'),
                'wind_direction': props.get('wind_from_direction')
            }
            rows.append(row)
        
        df = pd.DataFrame(rows)
        return df.sort_values('z_value', ascending=False)
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        raise

In [ ]:
def create_skewt_plot(data):
    """Create Skew-T plot from DataFrame"""
    # Convert units
    pressure = data['z_value'].values * units.hPa
    temperature = data['temperature'].values * units.degC
    dewpoint = data['dew_point_temperature'].values * units.degC
    wind_speed = data['wind_speed'].values * units.meter_per_second
    wind_dir = data['wind_direction'].values * units.degrees
    
    # Calculate wind components
    u, v = wind_components(wind_speed, wind_dir)

    # Create figure and SkewT object
    fig = plt.figure(figsize=(15, 10))
    skew = SkewT(fig, rotation=45)

    # Plot temperature and dewpoint
    skew.plot(pressure, temperature, 'r', linewidth=2, label='Temperature')
    skew.plot(pressure, dewpoint, 'g', linewidth=2, label='Dew Point')

    # Add wind barbs
    skew.plot_barbs(pressure, u, v)

    # Calculate and plot parcel profile
    prof = parcel_profile(pressure, temperature[0], dewpoint[0]).to('degC')
    skew.plot(pressure, prof, 'k', linewidth=1, linestyle='--', label='Parcel')

    # Add standard features
    skew.plot_dry_adiabats(alpha=0.4)
    skew.plot_moist_adiabats(alpha=0.4)
    skew.plot_mixing_lines(alpha=0.4)

    try:
        # Calculate stability indices
        cape, cin = cape_cin(pressure, temperature, dewpoint, parcel_profile_temperature=prof)
        li = lifted_index(pressure, temperature, parcel_profile_temperature=prof)
        pwat = precipitable_water(pressure, dewpoint)
        shr_6km = bulk_shear(pressure, u, v, depth=6000 * units.meter)

        # Add text box with calculated parameters
        info_txt = (
            f'CAPE: {cape.magnitude:.0f} J/kg\n'
            f'CIN: {cin.magnitude:.0f} J/kg\n'
            f'Lifted Index: {li.magnitude:.1f}\n'
            f'Precipitable Water: {pwat.magnitude:.1f} mm\n'
            f'6km Shear: {shr_6km.magnitude:.1f} m/s\n'
            f'Location: {data["lat"].iloc[0]:.2f}°N, {data["lon"].iloc[0]:.2f}°E\n'
            f'Time: {data["datetime"].iloc[0]}'
        )

        plt.text(0.85, 0.95, info_txt, transform=plt.gca().transAxes,
                bbox=dict(facecolor='white', alpha=0.7), fontsize=10,
                verticalalignment='top')

        # Shade CAPE and CIN areas
        skew.shade_cape(pressure, temperature, prof)
        skew.shade_cin(pressure, temperature, prof)
    except Exception as e:
        print(f"Warning: Could not calculate some stability indices: {e}")

    # Set limits and labels
    skew.ax.set_ylim(max(pressure.magnitude), min(pressure.magnitude))
    skew.ax.set_xlim(-40, 40)
    plt.title(f'Skew-T Log-P Diagram\nReport ID: {data["datetime"].iloc[0]}', fontsize=14)
    plt.legend(loc='upper right')
    skew.ax.grid(True, alpha=0.3)

    return fig

## Fetch and Plot Data

Now let's fetch the data and create the plot for a specific report ID:

In [ ]:
# Set the report ID
report_id = 271557

# Fetch the data
data = fetch_sounding_data(report_id)

# Display the first few rows of data
print("Raw data preview:")
display(data.head())

In [ ]:
# Create and display the plot
fig = create_skewt_plot(data)
plt.show()

## Analysis Functions

Let's add some additional analysis functions to examine the data:

In [ ]:
def analyze_sounding(data):
    """Perform basic analysis of the sounding data"""
    analysis = {
        'pressure_levels': len(data),
        'max_height': data['z_value'].min(),
        'surface_temp': data['temperature'].iloc[0],
        'surface_dewpoint': data['dew_point_temperature'].iloc[0],
        'max_wind': data['wind_speed'].max(),
        'temp_range': data['temperature'].max() - data['temperature'].min()
    }
    
    print("\nSounding Analysis:")
    print(f"Number of pressure levels: {analysis['pressure_levels']}")
    print(f"Maximum height (minimum pressure): {analysis['max_height']:.0f} hPa")
    print(f"Surface temperature: {analysis['surface_temp']:.1f}°C")
    print(f"Surface dewpoint: {analysis['surface_dewpoint']:.1f}°C")
    print(f"Maximum wind speed: {analysis['max_wind']:.1f} m/s")
    print(f"Temperature range: {analysis['temp_range']:.1f}°C")
    
    return analysis

In [ ]:
# Run the analysis
analysis = analyze_sounding(data)